In [8]:
import cv2
import numpy as np
import pickle

# --- 1. 画像読み込み（カラー）
img = cv2.imread("heavyrainfall_HR00040.png")
#img = cv2.imread(r"C:\Users\makok\mini_test\heavyrainfall_HR00044.png")


# --- 2. グレースケール変換
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# --- 3. 黒い線を抽出（閾値処理）
# 黒っぽい部分だけ抽出するために、上限値を低く
_, mask = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

# --- 4. ノイズ除去（細線や小さい点を除去）
kernel = np.ones((3, 3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)

# --- 5. 黒い線の輪郭抽出
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# --- 6. 「太さ」でフィルタ（県境など細線は除外）
selected_lines = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if min(w, h) > 2:  # 幅か高さがある程度太い場合のみ
        selected_lines.append(cnt)

# --- 7. 座標データをpklに保存
with open("thick_black_lines.pkl", "wb") as f:
    pickle.dump(selected_lines, f)

# --- 8. 抽出結果を確認用に描画
result = img.copy()
cv2.drawContours(result, selected_lines, -1, (0, 0, 255), 2)
cv2.imwrite("extracted_lines.png", result)


True

In [6]:
import os

path = "mini_test/heavyrainfall_HR00044.png"
print("Exists:", os.path.exists(path))

img = cv2.imread(path)
print("img is None:", img is None)


Exists: False
img is None: True


In [12]:
import cv2
import numpy as np
import pickle
import os

# ====== 入力設定 ======
img_path = "heavyrainfall_HR00040.png"
output_pkl = "thick_black_lines_geo.pkl"
output_img = "thick_black_lines_only.png"

# 中心座標（例: 実際の緯度経度に置き換え）
lat_c, lon_c = 35.0, 135.0
cx, cy = 500, 500  # 中心の画像上の位置
dlat_per_pixel = 0.01
dlon_per_pixel = 0.01
# =====================

# --- パス存在確認 ---
if not os.path.exists(img_path):
    raise FileNotFoundError(f"画像が見つかりません: {img_path}")

# --- 1. 画像読み込み ---
img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# --- 2. 黒い部分を抽出 ---
_, mask = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

# --- 3. ノイズ除去 ---
kernel = np.ones((3, 3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)

# --- 4. 輪郭検出 ---
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# --- 5. フィルタ（太さ＋面積） ---
selected_lines = []
selected_contours = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    area = cv2.contourArea(cnt)

    # 条件: 太さが2px以上 && 面積が50以上（凡例や細線除去）
    if min(w, h) > 2 and area > 50:
        geo_coords = []
        for point in cnt:
            px, py = point[0]
            lat = lat_c + (cy - py) * dlat_per_pixel
            lon = lon_c + (px - cx) * dlon_per_pixel
            geo_coords.append((lat, lon))
        selected_lines.append(geo_coords)
        selected_contours.append(cnt)

# --- 6. 線だけ描画（白背景） ---
line_img = np.ones_like(img) * 255
cv2.drawContours(line_img, selected_contours, -1, (0, 0, 0), thickness=2)
cv2.imwrite(output_img, line_img)

# --- 7. pkl保存 ---
with open(output_pkl, "wb") as f:
    pickle.dump(selected_lines, f)

print(f"黒い太線の座標を {output_pkl} に保存しました")
print(f"線のみの画像を {output_img} に保存しました")


黒い太線の座標を thick_black_lines_geo.pkl に保存しました
線のみの画像を thick_black_lines_only.png に保存しました


In [13]:
import cv2
import numpy as np
import pickle
import os

# ====== 入力設定 ======
img_path = "heavyrainfall_HR00040.png"
output_pkl = "thick_black_lines_geo_mew.pkl"
output_img = "thick_black_lines_only_new.png"

# 中心座標（例: 実際の緯度経度に置き換え）
lat_c, lon_c = 35.0, 135.0
cx, cy = 500, 500  # 中心の画像上の位置
dlat_per_pixel = 0.01
dlon_per_pixel = 0.01

# トリミング範囲（上下左右のpx）
trim_top = 20
trim_bottom = 50
trim_left = 20
trim_right = 80
# =====================

# --- パス存在確認 ---
if not os.path.exists(img_path):
    raise FileNotFoundError(f"画像が見つかりません: {img_path}")

# --- 1. 画像読み込み ---
img = cv2.imread(img_path)

# --- 2. トリミングで文字・凡例除去 ---
h, w = img.shape[:2]
img = img[trim_top:h-trim_bottom, trim_left:w-trim_right]

# 中心ピクセル座標もトリミング分を考慮
cx -= trim_left
cy -= trim_top

# --- 3. グレースケール＆黒抽出 ---
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

# --- 4. ノイズ除去 ---
kernel = np.ones((3, 3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)

# --- 5. 輪郭検出 ---
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# --- 6. 厳しめフィルタ（太さ + 面積 + 周長） ---
selected_lines = []
selected_contours = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, closed=False)

    # 条件:
    #   - 太さが3px以上
    #   - 面積が100px以上
    #   - 周長が50px以上
    if min(w, h) >= 3 and area > 100 and perimeter > 50:
        geo_coords = []
        for point in cnt:
            px, py = point[0]
            lat = lat_c + (cy - py) * dlat_per_pixel
            lon = lon_c + (px - cx) * dlon_per_pixel
            geo_coords.append((lat, lon))
        selected_lines.append(geo_coords)
        selected_contours.append(cnt)

# --- 7. 白背景に黒線のみ描画 ---
line_img = np.ones_like(img) * 255
cv2.drawContours(line_img, selected_contours, -1, (0, 0, 0), thickness=2)
cv2.imwrite(output_img, line_img)

# --- 8. pkl保存 ---
with open(output_pkl, "wb") as f:
    pickle.dump(selected_lines, f)

print(f"黒い太線の座標を {output_pkl} に保存しました")
print(f"線のみの画像を {output_img} に保存しました")


黒い太線の座標を thick_black_lines_geo_mew.pkl に保存しました
線のみの画像を thick_black_lines_only_new.png に保存しました


In [31]:
#うまくいったやつ

In [29]:
import cv2
import numpy as np
import pickle
import os

# --- 1. 画像読み込み
img = cv2.imread("heavyrainfall_HR00040.png")
if img is None:
    raise FileNotFoundError("画像が読み込めません。パスを確認してください。")
    
# トリミング範囲（上下左右のpx）
trim_top = 80
trim_bottom = 150
trim_left = 20
trim_right = 80

# --- 2. トリミングで文字・凡例除去 ---
h, w = img.shape[:2]
img = img[trim_top:h-trim_bottom, trim_left:w-trim_right]

# --- 2. グレースケール化
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# --- 3. 黒抽出（閾値）
_, mask = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

# --- 4. モルフォロジー処理でノイズ除去
kernel = np.ones((3, 3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)

# --- 5. 輪郭検出
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

filtered_contours = []
min_length = 100  # この長さ未満の線は除去

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    length = max(w, h)  # 長辺の長さ
    if length >= min_length:
        filtered_contours.append(cnt)

# --- 6. 黒線のみ描画
output = np.zeros_like(mask)
cv2.drawContours(output, filtered_contours, -1, 255, thickness=2)

# --- 7. pklとして保存
with open("black_lines_new.pkl", "wb") as f:
    pickle.dump(filtered_contours, f)

# --- 8. 保存
# 透明背景のRGBA画像作成
output_rgba = np.zeros((mask.shape[0], mask.shape[1], 4), dtype=np.uint8)

# 線部分を黒に、αを255にする
for cnt in filtered_contours:
    cv2.drawContours(output_rgba, [cnt], -1, (0,0,0,255), thickness=2)

cv2.imwrite("black_lines_only_transparent.png", output_rgba)

#cv2.imwrite("black_lines_only_new.png", output)

print(f"抽出した輪郭数: {len(filtered_contours)}")

# --- 9. その場で表示（matplotlib） ---
#plt.figure(figsize=(10,10))
#plt.imshow(line_img)
#plt.axis('off')
#plt.show()

# --- 9. OpenCVでその場で表示 ---
#cv2.imshow("Black Lines", line_img)   # line_img は白背景+黒線
#cv2.waitKey(0)                        # 任意のキーを押すまで待つ
#cv2.destroyAllWindows()               # ウィンドウを閉じる


抽出した輪郭数: 1


In [30]:
import cv2
import numpy as np
import pickle
import os

# --- 1. 画像読み込み
img = cv2.imread("heavyrainfall_HR00050.png")
if img is None:
    raise FileNotFoundError("画像が読み込めません。パスを確認してください。")
    
# トリミング範囲（上下左右のpx）
trim_top = 80
trim_bottom = 150
trim_left = 20
trim_right = 80

# --- 2. トリミングで文字・凡例除去 ---
h, w = img.shape[:2]
img = img[trim_top:h-trim_bottom, trim_left:w-trim_right]

# --- 2. グレースケール化
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# --- 3. 黒抽出（閾値）
_, mask = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

# --- 4. モルフォロジー処理でノイズ除去
kernel = np.ones((3, 3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)

# --- 5. 輪郭検出
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

filtered_contours = []
min_length = 100  # この長さ未満の線は除去

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    length = max(w, h)  # 長辺の長さ
    if length >= min_length:
        filtered_contours.append(cnt)

# --- 6. 黒線のみ描画
output = np.zeros_like(mask)
cv2.drawContours(output, filtered_contours, -1, 255, thickness=2)

# --- 7. pklとして保存
with open("black_lines_new_1.pkl", "wb") as f:
    pickle.dump(filtered_contours, f)

# --- 8. 保存
# 透明背景のRGBA画像作成
output_rgba = np.zeros((mask.shape[0], mask.shape[1], 4), dtype=np.uint8)

# 線部分を黒に、αを255にする
for cnt in filtered_contours:
    cv2.drawContours(output_rgba, [cnt], -1, (0,0,0,255), thickness=2)

cv2.imwrite("black_lines_only_transparent_1.png", output_rgba)

#cv2.imwrite("black_lines_only_new.png", output)

print(f"抽出した輪郭数: {len(filtered_contours)}")


抽出した輪郭数: 1
